In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import re as re
import time
import pandas as pd
from selenium.common.exceptions import NoSuchWindowException, WebDriverException,JavascriptException,TimeoutException,NoSuchElementException

In [3]:
PATH = input("/Users/thanuathitya/Scrapping/chromedriver-mac-x64/chromedriver")
print(PATH)


/Users/thanuathitya/Scrapping/chromedriver-mac-x64/chromedriver 


In [4]:
# Use driver to open the link
driver = webdriver.Chrome(PATH)
driver.get("https://www.linkedin.com/uas/login")
time.sleep(3)

In [5]:
# Creating lists
post_links = []
post_texts = []
post_names = []

In [6]:
def initialize_driver():
    # Initialize WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in headless mode
    options.add_argument('--disable-gpu')  # Disable GPU acceleration
    options.add_argument('--no-sandbox')  # Bypass OS security model
    options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
    return webdriver.Chrome(options=options)

def wait_for_element(driver, by, value, timeout=10):
    try:
        element_present = EC.presence_of_element_located((by, value))
        WebDriverWait(driver, timeout).until(element_present)
        return True
    except TimeoutException:
        return False

def Scrape_func(driver, a, b, c, post_limit):
    name = a.split("/")[-2]
    page = a
    time.sleep(10)

    try:
        driver.get(page + 'detail/recent-activity/shares/')
    except (NoSuchWindowException, WebDriverException) as e:
        print(f"Failed to load page: {page} - {e}")
        return
    
    start = time.time()
    retries = 3
    
    for _ in range(retries):
        try:
            if wait_for_element(driver, By.TAG_NAME, "body"):
                lastHeight = driver.execute_script("return document.body.scrollHeight")
                break
        except JavascriptException as e:
            print(f"Javascript error occurred: {e}")
            time.sleep(5)
    
    while True:
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        except JavascriptException as e:
            print(f"Javascript error during scroll: {e}")
            break
        
        time.sleep(5)
        
        try:
            newHeight = driver.execute_script("return document.body.scrollHeight")
        except JavascriptException as e:
            print(f"Javascript error occurred: {e}")
            break
        
        if newHeight == lastHeight:
            break
        lastHeight = newHeight
        end = time.time()
        if round(end - start) > 20:
            break

    company_page = driver.page_source
    linkedin_soup = bs(company_page.encode("utf-8"), "html.parser")
    linkedin_soup.prettify()
    containers = linkedin_soup.findAll("div", {"class": "occludable-update ember-view"})
    print(f"Fetching data from account: {name}")
    iterations = 0

    for container in containers:
        try:
            text_box = container.find("div", {"class": "feed-shared-update-v2__description-wrapper ember-view"})
            if not text_box:
                print("Text box not found")
                continue
            text = text_box.find("span", {"dir": "ltr"})
            if not text:
                print("Text span not found")
                continue
            b.append(text.text.strip())
            c.append(name)
            iterations += 1
            print(f"Post {iterations}: {text.text.strip()}")
            
            if iterations == post_limit:
                break
        except Exception as e:
            print(f"Error processing container: {e}")

In [7]:
driver = initialize_driver()

post_links = [
    "https://www.linkedin.com/in/sundarpichai/"
]

post_texts = []
post_names = []

try:
    n = int(input("Enter the number of entries: "))
    post_links = [input("Enter the link: ") for _ in range(n)]

    for link in post_links:
        try:
            Scrape_func(driver, link, post_texts, post_names, post_limit=5)
        except (NoSuchWindowException, WebDriverException) as e:
            print(f"Failed to process link: {link} - {e}")
            # Reinitialize the driver if window is closed
            driver.quit()
            driver = initialize_driver()

finally:
    driver.quit()


Enter the number of entries:  1
Enter the link:  https://www.linkedin.com/in/sundarpichai/


Fetching data from account: sundarpichai


In [8]:
data = {
    "Name": post_names,
    "Content": post_texts,
}

df = pd.DataFrame(data)
df.to_csv("test1.csv", encoding='utf-8', index=False)
